In [3]:
!pip install torch tensorflow numpy


import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import numpy as np

print("✅ 라이브러리 설치 및 임포트 완료!")

✅ 라이브러리 설치 및 임포트 완료!


In [31]:
class QNet(nn.Module):
    def __init__(self):
        super(QNet, self).__init__()
        self.fc1 = nn.Linear(5, 64)
        self.fc2 = nn.Linear(64, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

print("✅ QNet 클래스 정의 완료!")


✅ QNet 클래스 정의 완료!


In [32]:
q = QNet()
q.load_state_dict(torch.load("pong_model.pth"))
q.eval()

weights = {}
for name, param in q.named_parameters():
    weights[name] = param.detach().cpu().numpy()

print(f"레이어 목록: {list(weights.keys())}")
print("✅ PyTorch 모델 로드 후 가중치 추출 완료!")

레이어 목록: ['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias']
✅ PyTorch 모델 로드 후 가중치 추출 완료!


In [35]:
tf_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(5,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3)
])

tf_model.layers[0].set_weights([weights['fc1.weight'].T, weights['fc1.bias']])
tf_model.layers[1].set_weights([weights['fc2.weight'].T, weights['fc2.bias']])

print("✅ TensorFlow 모델 생성 및 가중치 복사 완료!")

✅ TensorFlow 모델 생성 및 가중치 복사 완료!


In [36]:
test_input = np.random.randn(1, 5).astype(np.float32)

with torch.no_grad():
    pytorch_output = q(torch.from_numpy(test_input)).numpy()

tf_output = tf_model.predict(test_input, verbose=0)

print(f"\n검증:")
print(f"PyTorch 출력:    {pytorch_output}")
print(f"TensorFlow 출력: {tf_output}")
print(f"차이:           {np.abs(pytorch_output - tf_output).max():.6f}")
print("✅ PyTorch vs TensorFlow 검증 완료!")


검증:
PyTorch 출력:    [[2.9104404 2.5798075 3.0080447]]
TensorFlow 출력: [[2.91044   2.5798073 3.008044 ]]
차이:           0.000001
✅ PyTorch vs TensorFlow 검증 완료!


In [37]:
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("pong_model.tflite", "wb") as f:
    f.write(tflite_model)
print("\n✅ TFLite 변환 완료!")

Saved artifact at '/tmp/tmpbgfhzpx9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  139207252566352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139207252564432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139207252561552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139207252561168: TensorSpec(shape=(), dtype=tf.resource, name=None)

✅ TFLite 변환 완료!


In [38]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], test_input)
interpreter.invoke()
tflite_output = interpreter.get_tensor(output_details[0]['index'])

print(f"\nTFLite 출력:     {tflite_output}")
print(f"차이(vs PyTorch): {np.abs(pytorch_output - tflite_output).max():.6f}")

from google.colab import files
files.download('pong_model.tflite')
print("\n✅ TFlite 변환 파일 다운로드 완료!")


TFLite 출력:     [[2.9104397 2.5798075 3.008045 ]]
차이(vs PyTorch): 0.000001


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ TFlite 변환 파일 다운로드 완료!
